In [1]:
import warnings
warnings.filterwarnings('ignore')

import gc
import glob
import ray
import sys
import os
import re
import cv2
import psutil
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.metrics import PQ
from utils.process_utils import remap_label
from utils.post_processing import *
from itertools import product
from collections import OrderedDict

# Filepaths

In [2]:
prob_map_pth = "/home/leos/IMAGE_ANALYSIS/model_ouputs/Kumar/probability_maps/V1/patch_224/unet_smp_seresnet50_kumar/orig/*.mat"
images_pth = "/home/leos/IMAGE_ANALYSIS/datasets/H&E/nuclei_segmentation/exhaustive_annotations/Nucleisegmentation-Kumar/test/Images/*.tif"
labels_pth = "/home/leos/IMAGE_ANALYSIS/datasets/H&E/nuclei_segmentation/exhaustive_annotations/Nucleisegmentation-Kumar/test/Labels/*.mat"

In [3]:
files = sorted(glob.glob(images_pth))
gt_masks = sorted(glob.glob(labels_pth))
prob_maps = sorted(glob.glob(prob_map_pth))

# check that ordering is the same 
for f, p, g in list(zip(files, prob_maps, gt_masks)):
    print(f.split('/')[-1], ", ",  p.split('/')[-1], ", ", g.split('/')[-1])

TCGA-2Z-A9J9-01A-01-TS1.tif ,  TCGA-2Z-A9J9-01A-01-TS1_prob_map_orig.mat ,  TCGA-2Z-A9J9-01A-01-TS1_mask.mat
TCGA-44-2665-01B-06-BS6.tif ,  TCGA-44-2665-01B-06-BS6_prob_map_orig.mat ,  TCGA-44-2665-01B-06-BS6_mask.mat
TCGA-69-7764-01A-01-TS1.tif ,  TCGA-69-7764-01A-01-TS1_prob_map_orig.mat ,  TCGA-69-7764-01A-01-TS1_mask.mat
TCGA-A6-6782-01A-01-BS1.tif ,  TCGA-A6-6782-01A-01-BS1_prob_map_orig.mat ,  TCGA-A6-6782-01A-01-BS1_mask.mat
TCGA-AC-A2FO-01A-01-TS1.tif ,  TCGA-AC-A2FO-01A-01-TS1_prob_map_orig.mat ,  TCGA-AC-A2FO-01A-01-TS1_mask.mat
TCGA-AO-A0J2-01A-01-BSA.tif ,  TCGA-AO-A0J2-01A-01-BSA_prob_map_orig.mat ,  TCGA-AO-A0J2-01A-01-BSA_mask.mat
TCGA-CU-A0YN-01A-02-BSB.tif ,  TCGA-CU-A0YN-01A-02-BSB_prob_map_orig.mat ,  TCGA-CU-A0YN-01A-02-BSB_mask.mat
TCGA-EJ-A46H-01A-03-TSC.tif ,  TCGA-EJ-A46H-01A-03-TSC_prob_map_orig.mat ,  TCGA-EJ-A46H-01A-03-TSC_mask.mat
TCGA-FG-A4MU-01B-01-TS1.tif ,  TCGA-FG-A4MU-01B-01-TS1_prob_map_orig.mat ,  TCGA-FG-A4MU-01B-01-TS1_mask.mat
TCGA-GL-6846-01A-01

# Run thresholding for all probability maps
- First we create the functions that are used in the pipeline 
- Second we create the parameter space
- Third we use ray to run the pipeline for different parameter settings in parallel

## 1. Create functions with ray.remote decorator that are used in the pipeline

In [4]:
# Garbage collector to empty memory
def auto_garbage_collect(pct=80.0):
    if psutil.virtual_memory().percent >= pct:
        gc.collect()

In [5]:
@ray.remote
def load_gt(fn):
    return scipy.io.loadmat(fn)['inst_map']


@ray.remote
def load_probmap(fn):
    return scipy.io.loadmat(fn)['prob_map']


@ray.remote
def activate(prob_map, method):
    return activation(prob_map, method)
    

@ray.remote
def thresh_naive(prob_map, thrsh, act, fn):
    thresh_mask = naive_thresh(prob_map, thrsh)
    return {
        'inst_map': thresh_mask, 
        'activation':act, 
        'threshold':thrsh, 
        'fn':fn.split('/')[-1]
    }


@ray.remote
def thresh_sauvola(prob_map, win_size, act, fn):
    thresh_mask = sauvola_thresh(prob_map, win_size)
    return {
        'inst_map': thresh_mask, 
        'activation':act, 
        'threshold':win_size, 
        'fn':fn.split('/')[-1]
    }


@ray.remote
def thresh_niblack(prob_map, win_size, act, fn):
    thresh_mask = niblack_thresh(prob_map, win_size)
    return {
        'inst_map': thresh_mask, 
        'activation':act, 
        'threshold':win_size, 
        'fn':fn.split('/')[-1]
    }


@ray.remote
def compute_PQ(true, pred, fn, key):
    metrics = PQ(true, pred)
    return {
        'metrics':metrics,
        'fn':fn,
        'key':key
    }


@ray.remote
def run_watershed(inst_map, win_size, fn):
    ws_mask = inv_dist_watershed(inst_map, win_size)
    return {
        'inst_map':ws_mask,
        'fn':fn
    }


@ray.remote
def run_random_walker(inst_map, contour, fn):
    rw_mask = random_walk(inst_map, contour)
    return {
        'inst_map':rw_mask,
        'fn':fn
    }

## 2. Create the parameter space

In [6]:
# Adjust either the thresholds or window sizes depending on what thresholding method youre using
# NOTE: too big parameter spaces will require lots of memory so make sure you have some

# thresholds = [3, 2.5, 2.2 ,2, 1.9, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3]
win_sizes = [num for num in range(13, 35) if num%2 != 0]
activations = ['sigmoid', 'relu']
combinations = list(product(*[activations, win_sizes, prob_maps]))

## 3. Run the thresholding pipeline
- Ray is used here to parallelize the pipeline

In [7]:
ray.init()

2020-06-25 16:39:09,712	INFO resource_spec.py:212 -- Starting Ray with 15.48 GiB memory available for workers and up to 7.74 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-25 16:39:10,077	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.214.129.142',
 'raylet_ip_address': '128.214.129.142',
 'redis_address': '128.214.129.142:38893',
 'object_store_address': '/tmp/ray/session_2020-06-25_16-39-09_711448_30689/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-25_16-39-09_711448_30689/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-25_16-39-09_711448_30689'}

### Thresholding pipeline

1. Load probability map
2. Run activation
3. Run thresholding (naive)

(Naive thresholding = try out different thresholds and find out which one works out the best)

NOTE: Other thresholding functions can also be used but they need to be added 

In [8]:
%%time

thresholded_masks = []
for act, thrsh, fn in combinations:
    prob_map = load_probmap.remote(fn)
    act_prob_map = activate.remote(prob_map, act)
    
    # Choose the tresholding function here
    #thresh_mask = thresh.remote(act_prob_map, thrsh, act, fn)
    #thresh_mask = thresh_niblack.remote(act_prob_map, thrsh, act, fn)
    thresh_mask = thresh_sauvola.remote(act_prob_map, thrsh, act, fn)
    thresholded_masks.append(thresh_mask)

thresholded_masks = ray.get(thresholded_masks)

CPU times: user 855 ms, sys: 230 ms, total: 1.09 s
Wall time: 13.3 s


### Group the obtained masks
- This is done only to make it easier to compute the PQ metrics
- Very quick operation

In [9]:
%%time

# Group the masks in separate dicts based on the parameter settings
masks_grouped = OrderedDict()
for act, thresh, _ in combinations:
    key = act + '_' + str(thresh)
    masks = []
    for mask in thresholded_masks:
        if mask['threshold'] == thresh and mask['activation'] == act:
            masks.append(mask)
    masks_grouped[key] = masks

CPU times: user 6.86 ms, sys: 0 ns, total: 6.86 ms
Wall time: 6.77 ms


# Compute PQ, SQ and DQ for every masks in every parameter setting
- Ray is used here again to parallelize computations

In [10]:
# Shut down ray to release memory
ray.shutdown()

In [11]:
del thresholded_masks
gc.collect()

0

In [12]:
# Start ray again
ray.init()

2020-06-25 16:39:24,898	INFO resource_spec.py:212 -- Starting Ray with 15.43 GiB memory available for workers and up to 7.73 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-25 16:39:25,212	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.214.129.142',
 'raylet_ip_address': '128.214.129.142',
 'redis_address': '128.214.129.142:43203',
 'object_store_address': '/tmp/ray/session_2020-06-25_16-39-24_897494_30689/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-25_16-39-24_897494_30689/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-25_16-39-24_897494_30689'}

In [13]:
%%time
PQs = []
for i, key in enumerate(masks_grouped.keys()):
    mask_group = masks_grouped[key]

    for mask, gt_fn in zip(mask_group, gt_masks):
        true = load_gt.remote(gt_fn)
        pred = mask['inst_map']
        fn = mask['fn']
        
        pq = compute_PQ.remote(true, pred, fn, key)
        PQs.append(pq)
        
PQs = ray.get(PQs)

CPU times: user 5.79 s, sys: 1.51 s, total: 7.3 s
Wall time: 2min 54s


### Group the PQ metrics based on the parameter setting
- This is again only done to make the downstream computations more straigth forward
- Quick to do

In [14]:
%%time

PQs_grouped = OrderedDict()
for key in masks_grouped.keys():
    PQ_group = []
    for d in PQs:
        if d['key'] == key:
            PQ_group.append(d['metrics'])
    PQs_grouped[key] = PQ_group

CPU times: user 394 µs, sys: 97 µs, total: 491 µs
Wall time: 494 µs


### Compute the averages of the metrics for each parameter setting
- the highest weighted sum is chosen as the best parameter setting

In [15]:
best = -np.Infinity
best_key = None
best_m = None
weights = np.array([1.2, 1.0, 1.1, 0.0, 0.0])

for key in PQs_grouped.keys():
    m = pd.DataFrame(PQs_grouped[key]).mean(axis=0)
    ws = m @ weights
    if ws > best:
        best = ws
        best_key = key
        best_m = m

print('best param setting for thresholding: ', best_key, '\n')
print('metrics averaged:\n', best_m, '\n', sep='')
print('weighted sum =', best)

best param setting for thresholding:  relu_21 

metrics averaged:
pq             0.511774
sq             0.720860
dq             0.708242
sensitivity    0.825767
precision      0.625327
dtype: float64

weighted sum = 2.1140554071370956


# Run post-processing segmentation pipeline for the masks that had highest averaged PQ in the thresholding part

In [16]:
del PQs_grouped
best_masks = masks_grouped[best_key]

In [17]:
%%time
metrics = []
result_masks = []

for mask in best_masks:
    # UNCOMMENT the segmentation method that is used 
    inst_map = run_watershed.remote(mask['inst_map'], 13, mask['fn'])
    # inst_map = run_random_walker.remote(mask['inst_map'], mask_contours(mask['inst_map']), mask['fn'])
    result_masks.append(inst_map)
result_masks = ray.get(result_masks)
    
for mask, gt_fn in zip(result_masks, gt_masks):
    gt = load_gt.remote(gt_fn)
    pq = compute_PQ.remote(gt, mask['inst_map'], mask['fn'], best_key)
    metrics.append(pq)    
metrics = ray.get(metrics)

(pid=4561) /home/leos/.local/lib/python3.6/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
(pid=4561)   def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
(pid=4922) /home/leos/.local/lib/python3.6/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
(pid=4922)   def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
(pid=4671) /home/leos/.local/lib/python3.6/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
(pid=4671)   def watershed(image, markers=None, connectivity=1, offset=None, mask=None,
(pid=4463) /home/leos/.lo

In [18]:
m = pd.DataFrame([m['metrics'] for m in metrics]).mean(axis=0)
ws = m @ weights

print('metrics averaged after segmentation:\n\n', m, '\n', sep='')
print('weighted sum =', ws)

metrics averaged after segmentation:

pq             0.501275
sq             0.719573
dq             0.695019
sensitivity    0.862479
precision      0.587216
dtype: float64

weighted sum = 2.085623995373284


# Save result masks

NOTE: Modify the filename variable according to your needs

In [19]:
# OUTPUT_DIR = "/home/leos/IMAGE_ANALYSIS/model_ouputs/Kumar/result_segmentations/V1/patch_224/unet_smp_seresnet50_kumar/hovernet_watershed_opening/Labels/"
# for mask in result_masks:
#     # modify the fn according to your needs
#     fn = mask['fn'].split('/')[-1].split('_')[0]
#     fn_mat = OUTPUT_DIR + fn + '_result_mask.mat'
#     inst_map = mask['inst_map']
#     print("write: ",  fn_mat)
#     scipy.io.savemat(fn_mat, mdict={'inst_map': inst_map})

In [20]:
ray.shutdown()